# UDN validation in Aetna DB

## Load the R libraries

In [ ]:
library("devtools")
library("SqlServerJtds")
library("SqlTools")
library("FactToCube")
library("dplyr")

## Set up the connection the the DB server

In [ ]:
cn = connect.sql.server(
    database="XXXX",
    domain="YYYY",
    server.address="ZZZZ",
    user= yourUserName,
    password= yourPassword)
cn

## Create the table to map from HPO to PheCode
Extracted from the Supplementary Material of: "Bastarache, L., Hughey, J.J., Hebbring, S., Marlo, J., Zhao, W., Ho, W.T., Van Driest, S.L., McGregor, T.L., Mosley, J.D., Wells, Q.S. and Temple, M., 2018. Phenotype risk scores identify patients with unrecognized Mendelian disease patterns. Science, 359(6381), pp.1233-1239."

### Mapping limitations
- Many of the mapped pairs were not exact matches. 
- Most often, the HPO term was more specific than the mapped phecode
(e.g. HPO term “neurogenic bladder” is mapped to the phecode for “functional disorders of bladder”)
- In some instances, the best available PheCode was narrower than its corresponding HPO term (e.g., the HPO term “retinopathy” is mapped to the phecode for “nondiabetic retinopathy”)

2,358 HPO terms did not map to phecodes, either because HPO term was too specific to be ascertained by claims data (e.g., “depletion of mitochondrial DNA in liver”)
too general (e.g., “abnormality of the kidney”)
pertained to non-pathogenic variation not typically captured in a medical setting (e.g., “thin clavicles”)

### Read the file that maps from HPO to PheCode

In [ ]:
hpoToPhecode <- read.delim("hpoToPhecode.txt", colClasses = "character")
hpoToPhecode <- hpoToPhecode[, c(6,2,3,4)]
colnames(hpoToPhecode) <- c("HpoCode", "HpoTerm", "PheCode", "PheCodeTerm")
dim(hpoToPhecode[ hpoToPhecode$PheCode == "", ])
hpoToPhecode <- hpoToPhecode[ hpoToPhecode$PheCode != "", ]

### Read the files mapping from PheCode to ICD9-CM and ICD10-CM
We extracted these files from https://phewascatalog.org/

In [ ]:
icd9tophewas <- read.delim("phecode_icd9_rolled.csv", 
                           colClasses = "character",
                           sep=",", 
                           header=TRUE)
icd9tophewas <- icd9tophewas[, c(1:3)]
colnames(icd9tophewas)[2] <- "ICD9description"

icd10tophewas <- read.delim("Phecode_map_v1_2_icd10cm_beta.csv", 
                           colClasses = "character",
                           sep=",", 
                           header=TRUE)
icd10tophewas <- icd10tophewas[, c(1:3)]
colnames(icd10tophewas) <- c("ICD10","ICD10description", "PheCode")

mappingHpoICD9 <- inner_join( icd9tophewas, hpoToPhecode)
mappingHpoICD9 <- mappingHpoICD9[, c(1,2,4,5,3,6)]
colnames(mappingHpoICD9) <- c("ICD", "ICDdescription", "HpoCode", "HpoDescription", "PheCode", "PheCodeDescription")
mappingHpoICD9$version <- 9

mappingHpoICD10 <- inner_join( icd10tophewas, hpoToPhecode)
mappingHpoICD10 <- mappingHpoICD10[, c(1,2,4,5,3,6)]
colnames(mappingHpoICD10) <- c("ICD", "ICDdescription", "HpoCode", "HpoDescription", "PheCode", "PheCodeDescription")
mappingHpoICD10$version <- 10

hpoToPhecodeToICD <- rbind( mappingHpoICD9, mappingHpoICD10)

### Create a SQL table with the mapping from HPO to ICD9 and ICD10 through PheCode
This mapping table contains:
- 875 PheCodes
- 3,403 HpoCodes
- 6,812 ICD9-CM codes
- 25,700 ICD10-CM codes

In [ ]:
dbSendUpdate( cn, "DROP TABLE XXX.dbo.hpoToPhecodeToICD")
dbWriteTable( cnag, "hpoToPhecodeToICD", hpoToPhecodeToICD, row.names = FALSE)

## 1. Aetna patients with at least one of the map codes
Create a temporal table with all the patients from 0 to 18 years old that have at least one ICD9 or ICD10 code that map to a HPO term. 

In [ ]:
dbSendUpdate(cn, "DROP TABLE #tmpAllIcd")
dbSendUpdate( cn, "SELECT DISTINCT F.MemberId, F.Icd, F.DateServiceStarted, M.Gender, M.BirthYear, (YEAR( F.DateServiceStarted ) - M.BirthYear) AS Age
INTO #tmpAllIcd
FROM FactIcd F
INNER JOIN Members M ON
F.MemberId=M.MemberId
WHERE
F.Icd IN (SELECT ICD FROM ag440.dbo.hpoToPhecodeToICD)
AND 
(YEAR( F.DateServiceStarted ) - M.BirthYear) < 18
AND 
(YEAR( F.DateServiceStarted ) - M.BirthYear) >= 0
AND IcdVersion IN ('9', '10')
")

dbSendUpdate( cn, "CREATE CLUSTERED INDEX idxPk ON #tmpAllIcd (MemberId, Icd, Age) WITH (DATA_COMPRESSION=PAGE)" )

## 2. Filtering by months of coverage
For the previous patients, select those that between 0 to 18 years old they have more than 12 months of coverage on the database with some ICD9 or ICD10 code and with the gender defined as M or F. 

In [ ]:
dbSendUpdate( cn, "SELECT
             E.MemberId, COUNT(DISTINCT E.EffectiveDate) AS nMonthsObservation
             INTO #tmpAllMemberCount
             FROM
             Enrollment E
             INNER JOIN Members M ON
             E.MemberId=M.MemberId
             INNER JOIN #tmpAllIcd C ON
             M.MemberId=C.MemberId
             WHERE
             E.MedicalIndicator = 'Y'
             AND (YEAR(E.EffectiveDate) - M.BirthYear) < 18
             AND (YEAR(E.EffectiveDate) - M.BirthYear) >= 0
             AND (M.Gender = 'F' OR M.Gender = 'M') 
             GROUP BY E.MemberId
             HAVING COUNT( DISTINCT E.EffectiveDate ) > 12")

## 3. Generate a table with members and ICD codes
Based on the two previous temporal tables generate a final table with those patients in common. We are only interested in the ICD codes, so we will not keep the rest of information. 

In [ ]:
dbSendUpdate( cn, "SELECT MemberId, Icd 
                    INTO ag440.dbo.completeHpoTable0to17
                    FROM #tmpAllIcd 
                    WHERE MemberId IN ( SELECT MemberId FROM #tmpAllMemberCount) GROUP BY MemberId, Icd")

## 4. Add the corresponding HPO term for each ICD code

In [ ]:
dbSendUpdate( cn, "DROP TABLE ag440.dbo.completeHpoTable0to17Mapped" )
dbSendUpdate( cn, "SELECT T.MemberId, T.Icd, M.HpoCode
                    INTO ag440.dbo.completeHpoTable0to17Mapped
                    FROM ag440.dbo.completeHpoTable0to17 T
                    INNER JOIN ag440.dbo.hpoToPhecodeToICD M ON T.Icd = M.ICD")

#remove duplicates
dbGetQuery( cn, "WITH CTE AS(
             SELECT *, ROW_NUMBER() 
             OVER ( PARTITION BY MemberId, Icd, HpoCode 
             ORDER BY MemberId, Icd, HpoCode) AS RN
             FROM
             ag440.dbo.completeHpoTable0to17Mapped
             )
             DELETE FROM CTE WHERE RN<>1" )

#create index
dbSendUpdate( cn, "CREATE CLUSTERED INDEX idxPk 
                    ON ag440.dbo.completeHpoTable0to17Mapped (MemberId, Icd, HpoCode) 
                    WITH (DATA_COMPRESSION=PAGE)" )

Once we have the tables created we can run our analysis. The objective is:
- check how many patients in Aetna have a list of 5 HPO codes that are the most prevalent in UDN identified clusters
- check if it is statistically significant by doing a bootstrp of rando HPO code selection

<section data-markdown>
                    <img src="./generalWorkflow.png" | width =500/>
                </section>
                
# Bootstrap
Random selection of 5 HPO terms and count:
- how many patients present the 5 terms
- how many patients present 4 of the terms
- how many patients present 3 of the terms
- how many patients present 2 of the terms
- how many patients present 1 of the terms

In [ ]:
# First we create an empty data frame
myoutputRandom <- as.data.frame( matrix( ncol=6, nrow=1000))
colnames( myoutputRandom ) <- c("randomHPO", "patientsHavingAll", "patientsHaving4", "patientsHaving3", "patientsHaving2", "patientsHaving1")

for( i in 1:1000){
    
    print( i )
    
    #random selection of 5 HPO codes
    dbSendUpdate( cn, "SELECT TOP 5 HpoCode INTO #randomHpo FROM ag440.dbo.hpoToPhecodeToICD ORDER BY NEWID()")
    
    #extract the 5 HPO codes
    hpoterms <- dbGetQuery(cn, "SELECT * FROM #randomHpo ")
    
    #save them in the first column of our dataframe
    myoutputRandom$randomHPO[i] <- paste( hpoterms$HpoCode, collapse = " - " )
    
    
    myoutputRandom$patientsHavingAll[i] <- nrow(dbGetQuery( cn, "SELECT MemberId, COUNT( DISTINCT HpoCode) AS nHPO 
                                                                    FROM ag440.dbo.completeHpoTable0to17Mapped 
                                                                    WHERE HpoCode IN 
                                                                    (SELECT HpoCode FROM #randomHpo) 
                                                                    GROUP BY MemberId 
                                                                    HAVING COUNT( DISTINCT HpoCode) = 5"))

   myoutputRandom$patientsHaving4[i] <- nrow(dbGetQuery( cn, "SELECT MemberId, COUNT( DISTINCT HpoCode) AS nHPO 
                                                                    FROM ag440.dbo.completeHpoTable0to17Mapped 
                                                                    WHERE HpoCode IN 
                                                                    (SELECT HpoCode FROM #randomHpo) 
                                                                    GROUP BY MemberId 
                                                                    HAVING COUNT( DISTINCT HpoCode) = 4"))
    
    
    myoutputRandom$patientsHaving3[i] <- nrow(dbGetQuery( cn, "SELECT MemberId, COUNT( DISTINCT HpoCode) AS nHPO 
                                                                    FROM ag440.dbo.completeHpoTable0to17Mapped 
                                                                    WHERE HpoCode IN 
                                                                    (SELECT HpoCode FROM #randomHpo) 
                                                                    GROUP BY MemberId 
                                                                    HAVING COUNT( DISTINCT HpoCode) = 3"))
    
    myoutputRandom$patientsHaving2[i] <- nrow(dbGetQuery( cn, "SELECT MemberId, COUNT( DISTINCT HpoCode) AS nHPO 
                                                                    FROM ag440.dbo.completeHpoTable0to17Mapped 
                                                                    WHERE HpoCode IN 
                                                                    (SELECT HpoCode FROM #randomHpo) 
                                                                    GROUP BY MemberId 
                                                                    HAVING COUNT( DISTINCT HpoCode) = 2"))
    
    myoutputRandom$patientsHaving1[i] <- nrow(dbGetQuery( cn, "SELECT MemberId, COUNT( DISTINCT HpoCode) AS nHPO 
                                                                    FROM ag440.dbo.completeHpoTable0to17Mapped 
                                                                    WHERE HpoCode IN 
                                                                    (SELECT HpoCode FROM #randomHpo) 
                                                                    GROUP BY MemberId 
                                                                    HAVING COUNT( DISTINCT HpoCode) = 1"))
    dbSendUpdate( cn, "DROP TABLE #randomHpo")

}

#save the data frame with the HPO codes and the counts
save(myoutputRandom, file="1000randomHPOselectionNumbers.RData")